## Import Libraries

In [1]:
import requests
from requests.auth import HTTPBasicAuth

import csv
import time
import glob

try:
  import pandas as pd
except ModuleNotFoundError:
  !pip install pandas
  import pandas as pd

from datetime import datetime, timezone


## Set Global Variables

In [ ]:
# Planning centre URLs
BASE_URL_SERVICES = 'https://api.planningcenteronline.com/services/v2/'
PEOPLE_BASE_URL = 'https://api.planningcenteronline.com/people/v2/people/'
CAMPUS_BASE_URL = 'https://api.planningcenteronline.com/people/v2/campuses'
BASE_URL_WORKFLOW = 'https://api.planningcenteronline.com/people/v2/workflows/'

# Planning centre API
API_APP_ID = " "
API_SECRET = " "

## OPTIONAL: Retrieve all the workflow IDs

In [3]:
# Function to fetch all workflow IDs
def get_all_workflow_ids():
    workflow_ids = []
    url = BASE_URL_WORKFLOW

    while url:
        response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

        if response.status_code == 200:
            workflows_data = response.json()

            # Extract workflow IDs from the current page
            for workflow in workflows_data['data']:
                workflow_ids.append(workflow['id'])

            # Check if there is a next page
            url = workflows_data['links'].get('next')
        else:
            print(f"Error fetching workflows: {response.status_code}, {response.json()}")
            break

    return workflow_ids

if __name__ == "__main__":
    workflow_ids = get_all_workflow_ids()
    print(f"Total Workflows: {len(workflow_ids)}")
    print("Workflow IDs:", workflow_ids)

Total Workflows: 35
Workflow IDs: ['548715', '544795', '544778', '564585', '544755', '544593', '550431', '550408', '544671', '544864', '544777', '544842', '550389', '541922', '550371', '544885', '544809', '564649', '548180', '544664', '550421', '544725', '550386', '544846', '555089', '564646', '548148', '544649', '550418', '544718', '550368', '544881', '544844', '550397', '561167']


## Retrieve information on all members

In [4]:
def fetch_all_people(per_page=100):
    url = f'{PEOPLE_BASE_URL}'
    all_results = []
    offset = 0

    while True:
        # Fetch a batch of data
        response = requests.get(f'{url}?per_page={per_page}&offset={offset}', auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

        if response.status_code == 200:
            data = response.json()
            if not data['data']:  # No more data to fetch
                break

            all_results.extend(data['data'])  # Append results to the list

            # Update offset to fetch the next batch
            offset += per_page
        else:
            print(f"Error fetching data: {response.status_code}, {response.json()}")
            break

    return all_results


def save_people_data_to_csv(people_data, csv_filename='people_data.csv'):
    """
    Saves a list of people data to a CSV file.

    Args:
        people_data (list): A list of dictionaries containing people data.
        csv_filename (str): The name of the CSV file to save the data to.
    """
    # Define the columns for the CSV
    csv_columns = [
        'id', 'first_name', 'last_name', 'name', 'birthdate', 'gender', 'membership',
        'status', 'created_at', 'updated_at', 'avatar', 'accounting_administrator',
        'can_create_forms', 'can_email_lists', 'child', 'passed_background_check',
        'people_permissions', 'site_administrator', 'primary_campus_id'
    ]

    # Open CSV file for writing
    try:
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()

            # Write each person entry to the CSV file
            for person in people_data:
                attributes = person['attributes']
                relationships = person['relationships']

                row = {
                    'id': person['id'],
                    'first_name': attributes.get('first_name'),
                    'last_name': attributes.get('last_name'),
                    'name': attributes.get('name'),
                    'birthdate': attributes.get('birthdate'),
                    'gender': attributes.get('gender'),
                    'membership': attributes.get('membership'),
                    'status': attributes.get('status'),
                    'created_at': attributes.get('created_at'),
                    'updated_at': attributes.get('updated_at'),
                    'avatar': attributes.get('avatar'),
                    'accounting_administrator': attributes.get('accounting_administrator'),
                    'can_create_forms': attributes.get('can_create_forms'),
                    'can_email_lists': attributes.get('can_email_lists'),
                    'child': attributes.get('child'),
                    'passed_background_check': attributes.get('passed_background_check'),
                    'people_permissions': attributes.get('people_permissions'),
                    'site_administrator': attributes.get('site_administrator'),
                    #'primary_campus_id':relationships.get('primary_campus', {}).get('data', {}).get('id')
                }

                writer.writerow(row)

        print(f"CSV file '{csv_filename}' created successfully.")

    except IOError:
        print("I/O error while writing the CSV file.")

people = fetch_all_people()
save_people_data_to_csv(people_data=people)

# Load the CSV file into a DataFrame
people_df = pd.read_csv('people_data.csv')

CSV file 'people_data.csv' created successfully.


## Get Connect Group data

In [ ]:

def get_person_name(person_id):
    """
    Function to get a person's name using their person_id from a CSV file.

    Args:
        person_id (str): The ID of the person.

    Returns:
        str: The name of the person, or "Unknown Person" if not found.
    """
    # Filter the DataFrame to find the row matching the person_id
    person = people_df[people_df['id'].astype(str) == person_id]

    if not person.empty:
        return person.iloc[0]['name']
    else:
        return "Unknown Person"

def get_person_email(person_id):
    """
    Function to get a person's email using their person_id from a CSV file.

    Args:
        person_id (str): The ID of the person.

    Returns:
        str: The email of the person, or "No Email Found" if not found.
    """
    # Filter the DataFrame to find the row matching the person_id
    person = people_df[people_df['id'].astype(str) == person_id]

    if not person.empty:
        # Assuming 'email' column contains the email, adjust if different
        return person.iloc[0].get('email', "No Email Found")
    else:
        return "No Email Found"

def get_person_email(person_id):
    url = f'{PEOPLE_BASE_URL}{person_id}/emails'
    response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

    if response.status_code == 200:
        emails_data = response.json()
        if emails_data['data']:
            primary_email = next((email['attributes']['address'] for email in emails_data['data'] if email['attributes']['primary']), emails_data['data'][0]['attributes']['address'])
            return primary_email
        else:
            return "No Email Found"
    else:
        print(f"Error fetching email for person {person_id}: {response.status_code}, {response.json()}")
        return "Unknown Email"

def get_person_email(person_id, retries=5, batch_size=100):
    """
    Function to get a person's email using their person_id, with pagination using offset.

    Args:
        person_id (str): The ID of the person.
        retries (int): The number of retries in case of rate-limiting.
        batch_size (int): The number of records per batch (default is 100).

    Returns:
        str: The primary email of the person, or "No Email Found" if not found.
    """
    offset = 0
    all_emails = []

    for attempt in range(retries):
        while True:
            url = f'{PEOPLE_BASE_URL}{person_id}/emails?offset={offset}&per_page={batch_size}'
            response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

            if response.status_code == 200:
                emails_data = response.json()

                # Append all emails fetched in this batch
                all_emails.extend(emails_data['data'])

                # If there are no more emails to fetch, break the loop
                if not emails_data['data'] or 'next' not in emails_data['links']:
                    break

                # Move to the next page by updating the offset
                offset += batch_size

            elif response.status_code == 429:  # Too Many Requests
                retry_after = response.headers.get('Retry-After')
                if retry_after:
                    wait_time = int(retry_after)
                else:
                    wait_time = 2 ** attempt  # Exponential backoff if Retry-After is not provided
                print(f"Rate limit exceeded. Waiting {wait_time} seconds before retrying...")
                time.sleep(wait_time)

            else:
                print(f"Error fetching email for person {person_id}: {response.status_code}, {response.json()}")
                return "No Email Found"

    # After fetching all pages, find the primary email or return the first email found
    if all_emails:
        primary_email = next((email['attributes']['address'] for email in all_emails if email['attributes']['primary']), all_emails[0]['attributes']['address'])
        return primary_email
    else:
        return "No Email Found"
# Function to get the list of workflow steps
def get_workflow_steps(workflow_id):
    url = f'{BASE_URL_WORKFLOW}{workflow_id}/steps'
    response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching steps for workflow {workflow_id}: {response.status_code}, {response.json()}")
        return None

# Function to get all people (cards) in the workflow
def get_workflow_people(workflow_id, retries=5, batch_size=100):
    """
    Fetches all people (workflow cards) from the specified workflow using pagination with offset.

    Args:
        workflow_id (str): The ID of the workflow.
        retries (int): The number of retries in case of rate-limiting or network issues.
        batch_size (int): The number of people to fetch per batch (default is 100).

    Returns:
        list: A list of all people (cards) in the workflow, or None if an error occurs.
    """
    offset = 0
    url = f'{BASE_URL_WORKFLOW}/{workflow_id}/cards'
    all_people = []

    for attempt in range(retries):
        while True:
            # Fetch a batch of people using pagination with offset
            response = requests.get(f'{url}?offset={offset}&per_page={batch_size}', auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

            if response.status_code == 200:
                people_data = response.json()
                all_people.extend(people_data['data'])  # Append the people data to the result list

                # If no more data, break out of the loop
                if not people_data['data'] or 'next' not in people_data['links']:
                    break

                # Move to the next page by updating the offset
                offset += batch_size

            elif response.status_code == 429:  # Too Many Requests
                retry_after = response.headers.get('Retry-After')
                if retry_after:
                    wait_time = int(retry_after)
                else:
                    wait_time = 2 ** attempt  # Exponential backoff if Retry-After is not provided
                print(f"Rate limit exceeded. Waiting {wait_time} seconds before retrying...")
                time.sleep(wait_time)

            else:
                print(f"Error fetching people for workflow {workflow_id}: {response.status_code}, {response.json()}")
                return None

        return all_people  # Return the list of all people after fetching all pages

    print(f"Failed to fetch workflow people after {retries} retries.")
    return None

# Function to calculate how long someone has been in a step
def calculate_days_in_step(moved_to_step_at):
    moved_to_step_at_datetime = datetime.strptime(moved_to_step_at, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc)
    now = datetime.now(timezone.utc)
    days_in_step = (now - moved_to_step_at_datetime).days
    return days_in_step

# Function to get the campus data and map campus IDs to names
def get_campus_mapping():
    url = CAMPUS_BASE_URL
    response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

    if response.status_code == 200:
        campus_data = response.json()
        campus_mapping = {campus['id']: campus['attributes']['name'] for campus in campus_data['data']}
        return campus_mapping
    else:
        print(f"Error fetching campuses: {response.status_code}, {response.json()}")
        return {}

def get_card_id_for_person_in_workflow(workflow_id, person_id, retries=5, per_page=100):
    """
    Function to get the card ID for a person in a workflow with pagination and retry mechanism.

    Args:
        workflow_id (str): The ID of the workflow.
        person_id (str): The ID of the person.
        retries (int): Number of retry attempts in case of rate-limiting.
        per_page (int): Number of results to fetch per page (default is 100).

    Returns:
        str: The card ID for the person, or None if not found.
    """
    offset = 0
    url = f'{BASE_URL_WORKFLOW}/{workflow_id}/cards'

    for attempt in range(retries):
        while True:
            # Fetch a batch of workflow cards using pagination with offset
            response = requests.get(f'{url}?per_page={per_page}&offset={offset}', auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

            if response.status_code == 200:
                cards_data = response.json()

                # Search for the card with the matching person_id
                for card in cards_data['data']:
                    card_person_id = card['relationships']['person']['data']['id']
                    if card_person_id == person_id:
                        return card['id']  # This is the card_id

                # If there are no more cards, break out of the loop
                if not cards_data['data'] or 'next' not in cards_data['links']:
                    break

                # Move to the next page
                offset += per_page

            elif response.status_code == 429:  # Too Many Requests
                # Check if a "Retry-After" header is provided
                retry_after = response.headers.get('Retry-After')
                if retry_after:
                    wait_time = int(retry_after)
                else:
                    wait_time = 2 ** attempt  # Exponential backoff if Retry-After is not provided
                print(f"Rate limit exceeded. Waiting {wait_time} seconds before retrying...")
                time.sleep(wait_time)

            else:
                print(f"Error fetching workflow cards: {response.status_code}, {response.json()}")
                return None

        print(f"No card found for person ID: {person_id}")
        return None

    print(f"Failed to fetch workflow cards after {retries} retries.")
    return None

# Function to get activities for a workflow card and filter by "completed the step"
def get_workflow_step_history_from_activities(workflow_id, card_id):
    url = f'{BASE_URL_WORKFLOW}/{workflow_id}/cards/{card_id}/activities' #Changed WORKFLOW
    response = requests.get(url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

    if response.status_code == 200:
        activities_data = response.json()
        activities = activities_data['data']
        step_history = [activity for activity in activities]

        return step_history
    else:
        print(f"Error fetching activities: {response.status_code}, {response.json()}")
        return None

# Function to retrieve step history for CSV export
def get_step_history_for_csv(workflow_id, person_id):
    card_id = get_card_id_for_person_in_workflow(workflow_id, person_id)
    if card_id:
        step_history = get_workflow_step_history_from_activities(workflow_id, card_id)
        history_str = ""
        if step_history:
            for activity in step_history:
                comment = activity['attributes']['comment']
                created_at = activity['attributes']['created_at']
                history_str += f"{comment} on {created_at}; "
        return history_str.strip("; ")
    return "No step history"

# Main function to list people by workflow step, with additional fields and export to CSV
def list_people_by_workflow_step_to_csv(workflow_id, csv_filename='workflow_data.csv'):

    workflow_steps = get_workflow_steps(workflow_id)
    workflow_info_url = f'{BASE_URL_WORKFLOW}{workflow_id}'

    # Fetch campus mapping
    campus_mapping = get_campus_mapping()

    # Fetch workflow info to get the name and campus ID
    workflow_info_response = requests.get(workflow_info_url, auth=HTTPBasicAuth(API_APP_ID, API_SECRET))

    if workflow_info_response.status_code == 200:
        workflow_info = workflow_info_response.json()
        workflow_name = workflow_info['data']['attributes']['name']

        try:
          campus_id = workflow_info['data']['relationships'].get('campus', {}).get('data', {}).get('id')
          campus_name = campus_mapping.get(campus_id, 'Unknown Campus')
        except:
          campus_name = 'Unknown Campus'

    else:
        print(f"Error fetching workflow info: {workflow_info_response.status_code}")
        return

    if workflow_steps:
        steps_dict = {step['id']: step['attributes']['name'] for step in workflow_steps['data']}
        people_in_workflow = get_workflow_people(workflow_id)

        if people_in_workflow:
            data_rows = []

            for person_card in people_in_workflow:
                person_id = person_card['relationships']['person']['data']['id']
                assignee_id = person_card['relationships'].get('assignee', {}).get('data', {}).get('id')
                try:
                  workflow_step = steps_dict[current_step_id]
                except:
                  workflow_step = None

                try:
                  current_step_id = person_card['relationships']['current_step']['data']['id']
                except:
                  current_step_id = None
                person_name = get_person_name(person_id)

                # Get assignee name and email
                assignee_name = get_person_name(assignee_id) if assignee_id else "Unknown Assignee"
                assignee_email = get_person_email(assignee_id) if assignee_id else "Unknown Email"

                # Get the date they were moved to the current step
                moved_to_step_at = person_card['attributes'].get('moved_to_step_at', None)
                removed_at = person_card['attributes'].get('removed_at', None)
                completed_at = person_card['attributes'].get('completed_at', None)

                # Determine removal status and calculate days in step
                if moved_to_step_at:
                    days_in_step = calculate_days_in_step(moved_to_step_at)
                else:
                    days_in_step = "Unknown"

                # If removed, show the removed date
                if removed_at:
                    removed_at_datetime = datetime.strptime(removed_at, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc)
                    removed = True
                    date_removed = removed_at_datetime.strftime('%Y-%m-%d')
                else:
                    removed = False
                    date_removed = "N/A"

                # Fetch step history for the person
                step_history = get_step_history_for_csv(workflow_id, person_id)

                # Append data for CSV
                data_rows.append({
                    'Campus Name': campus_name,
                    'Workflow Name': workflow_name,
                    'Workflow Step': workflow_step,
                    'Workflow Step ID': current_step_id,
                    'Person Name': person_name,
                    'Assignee Name': assignee_name,
                    'Assignee Email': assignee_email,
                    'Moved to Step': moved_to_step_at if moved_to_step_at else "Unknown",
                    'Completed_at': completed_at,
                    'Removed': removed,
                    'Date Removed': date_removed,
                    'Days in Step': days_in_step,
                    'Step History': step_history
                })

            # Write data to CSV
            csv_columns = ['Campus Name', 'Workflow Name', 'Workflow Step','Workflow Step ID', 'Person Name', 'Assignee Name', 'Assignee Email', 'Moved to Step', 'Completed_at','Removed', 'Date Removed', 'Days in Step', 'Step History']
            try:
                with open(csv_filename, 'w', newline='') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                    writer.writeheader()
                    writer.writerows(data_rows)
                print(f"CSV file '{csv_filename}' created successfully.")
            except IOError:
                print("I/O error while writing the CSV file.")
        else:
            print("No people found in the workflow.")
    else:
        print("No steps found for this workflow.")

In [6]:
workflow_ids = ['548715', '544795', '544778', '564585', '544755', '544593', '550431', '550408', '544671', '544864', '544777', '544842', '550389', '541922', '550371', '544885', '544809', '564649', '548180', '544664', '550421', '544725', '550386', '544846', '555089', '564646', '548148', '544649', '550418', '544718', '550368', '544881', '544844', '550397', '561167']


for workflow_id in workflow_ids:
  list_people_by_workflow_step_to_csv(workflow_id, csv_filename=f'workflow_data_{workflow_id}.csv')


CSV file 'workflow_data_548715.csv' created successfully.
CSV file 'workflow_data_544795.csv' created successfully.
CSV file 'workflow_data_544778.csv' created successfully.
CSV file 'workflow_data_564585.csv' created successfully.
CSV file 'workflow_data_544755.csv' created successfully.
Error fetching activities: 429, {'errors': [{'code': '429', 'title': 'Too May Requests', 'detail': 'Rate limit exceeded: 101 of 100 requests per 20 seconds'}]}
Rate limit exceeded. Waiting 1 seconds before retrying...


KeyboardInterrupt: 

## Combine and Clean up files

In [ ]:
def combine_csv_files(output_filename='combined_workflow_data.csv'):
    # Use glob to find all CSV files matching the pattern
    csv_files = glob.glob('workflow_data_*.csv')

    # List to hold all DataFrames
    dataframes = []

    # Loop through all matching CSV files
    for csv_file in csv_files:
        try:
            # Read each CSV file and append to the list of dataframes
            df = pd.read_csv(csv_file)
            dataframes.append(df)
            print(f"Loaded {csv_file} successfully.")
        except Exception as e:
            print(f"Error loading {csv_file}: {e}")

    # Concatenate all dataframes into one
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)

        # Save the combined dataframe to a new CSV file
        combined_df.to_csv(output_filename, index=False)
        print(f"Combined CSV saved to {output_filename}")
    else:
        print("No CSV files found or loaded.")

# Call the function to combine CSV files
combine_csv_files('combined_workflow_data.csv')

In [ ]:
def fill_missing_workflow_data(df, step_column='Workflow Step', id_column='Workflow Step ID'):
    """
    Function to fill missing Workflow Step or Workflow Step ID based on existing data in other rows.

    Args:
    df (pd.DataFrame): The DataFrame containing the workflow data.
    step_column (str): The name of the Workflow Step column.
    id_column (str): The name of the Workflow Step ID column.

    Returns:
    pd.DataFrame: DataFrame with missing Workflow Step or Workflow Step ID filled in.
    """
    # Step 1: Identify the complete pairs (rows where both columns are present)
    complete_pairs = df.dropna(subset=[step_column, id_column]).drop_duplicates(subset=[step_column, id_column])

    # Create a dictionary mapping Workflow Step to Workflow Step ID and vice versa
    step_to_id = dict(zip(complete_pairs[step_column], complete_pairs[id_column]))
    id_to_step = dict(zip(complete_pairs[id_column], complete_pairs[step_column]))

    # Step 2: Fill missing Workflow Step using Workflow Step ID
    df[step_column] = df.apply(
        lambda row: id_to_step.get(row[id_column], row[step_column]) if pd.isna(row[step_column]) else row[step_column],
        axis=1
    )

    # Step 3: Fill missing Workflow Step ID using Workflow Step
    df[id_column] = df.apply(
        lambda row: step_to_id.get(row[step_column], row[id_column]) if pd.isna(row[id_column]) else row[id_column],
        axis=1
    )

    return df

# Example usage:
df = pd.read_csv('combined_workflow_data.csv')

# Call the function to fill in missing data
filled_df = fill_missing_workflow_data(df)

# Display the result
filled_df.head()
filled_df.to_csv('filled_workflow_data.csv', index=False)